In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from statannot import add_stat_annotation
import matplotlib
import warnings
%matplotlib inline
warnings.simplefilter('ignore')
from arivale_data_interface import *
frozen_ss_path='/proj/arivale/snapshots/arivale_snapshot_ISB_2020-03-16_2156'
sn=list_snapshot_contents() #list all content
def get_frozen_snapshot(ss_name, ss_path=frozen_ss_path): #collect data
    return get_snapshot(ss_name, path=ss_path)
from statsmodels.stats.anova import anova_lm
import statsmodels.formula.api as smf
import scipy.stats
warnings.simplefilter('ignore')

In [2]:
genpath = '/proj/gibbons/kramos/github/IBT-and-the-Gut-Microbiome'

In [3]:
######### Import, clean, and format Arivale dataset #########

arivale=pd.read_csv(genpath+'/arivale_cohort/alpha-diversity.csv').rename(columns={'Unnamed: 0':'id'})

# Get sample metadata and merge
metadata = pd.read_csv('/proj/arivale/microbiome/16S_processed/metadata.csv')[['public_client_id', 'id', 'days_in_program','sex', 'age']]\
                        .rename(columns = {'public_client_id':'sample_id'})
arivale = pd.merge(arivale, metadata, on = 'id')

# Get weight data and merge
weights = get_frozen_snapshot('weight')[['public_client_id', 'days_in_program', 'BMI_CALC', 'HEIGHT_CALC']]\
                     .dropna().rename(columns = {'public_client_id':'sample_id'})
arivale = pd.merge_asof(
    arivale[arivale.sample_id.isin(weights.sample_id)].sort_values(by='days_in_program'),
    weights.sort_values(by='days_in_program'), 
    by = 'sample_id', on = 'days_in_program', direction='nearest')

# convert height from in to cm 
arivale['height_cm']=arivale['HEIGHT_CALC'].apply(lambda x: x*2.54)

# Finally, I need bowel movement frequency 
digestion = get_frozen_snapshot('assessments_digestive_health')[['public_client_id', 'days_in_program', 'assessment:digestion:bowel-movements:enum']].dropna().rename(columns = {'public_client_id':'sample_id'})

# Get the bowel movement number 
digestion['bowel_movement_frequency'] = digestion['assessment:digestion:bowel-movements:enum'].astype(str).apply(lambda x: x[1])
digestion['bowel_movement_frequency'] = digestion['bowel_movement_frequency'].astype(int)

arivale = pd.merge_asof(arivale.sort_values(by='days_in_program'),
    digestion.sort_values(by='days_in_program'),
    by='sample_id', on='days_in_program', direction='nearest')

# Add in dietary data
diet=get_frozen_snapshot('assessments')[['public_client_id', 'days_in_program', 'assessment:lifestyle:vegetables:enum']].dropna().reset_index(drop=True)\
.rename(columns={'assessment:lifestyle:vegetables:enum':'vegetable_frequency', 'public_client_id':'sample_id'})

# Make vegetable frequency numeric 
diet_num = list()
for i in range(0, len(diet)): 
    check = diet.iloc[i, 2]
    if check == '(0) Zero/less than 1 per day':
        diet_num.append(1)
    elif check == '(1) 1': 
        diet_num.append(2)
    elif check == '(2) 2-3': 
        diet_num.append(3)
    elif check == '(3) 4-5':
        diet_num.append(4)
    elif check == '(4) 6 or more': 
        diet_num.append(5) 
    else: 
        float('nan')
diet['vegetable_frequency']=diet_num
high = [5,4,3]
diet['b_vegetable_frequency'] = diet['vegetable_frequency'].apply(lambda x: 'high' if x in high else 'low')

arivale=pd.merge_asof(
    arivale.sort_values(by='days_in_program'), 
    diet[diet.sample_id.isin(arivale.sample_id)].sort_values(by='days_in_program'), 
    by='sample_id', on='days_in_program', direction='nearest')

arivale.head()

,Unnamed: 0.1,id,simpson,inv_simpson,sample_id,days_in_program,sex,age,BMI_CALC,HEIGHT_CALC,height_cm,assessment:digestion:bowel-movements:enum,bowel_movement_frequency,vegetable_frequency,b_vegetable_frequency
0,5216,AV15-4372|AKE035,0.973573,37.839776,01602320,-2.0,M,47.0,37.229730,74.0,187.96,(3) 1-3 times daily,3.0,1.0,low
1,5081,AV15-3716|AKE025,0.977155,43.774041,01228476,0.0,F,50.0,22.462722,65.0,165.10,(3) 1-3 times daily,3.0,4.0,high
2,4956,AV15-3489|AKE025,0.874632,7.976489,01826852,1.0,F,35.0,24.541709,61.0,154.94,(3) 1-3 times daily,3.0,4.0,high
3,5150,AV15-3840|AKE025,0.957654,23.615008,01602518,1.0,F,49.0,34.154541,64.0,162.56,(3) 1-3 times daily,3.0,2.0,low
4,5020,AV15-3636|AKE025,0.981430,53.849198,01447446,1.0,F,56.0,21.950816,70.0,177.80,(2) 3-6 times per week,2.0,3.0,high


In [4]:
# Get health markers from blood chemistries
chemistries = get_frozen_snapshot("chemistries")[['public_client_id', 'days_in_program', 'LDL-CHOL CALCULATION','CRP HIGH SENSITIVITY','GLYCOHEMOGLOBIN A1C']].sort_values(by='days_in_program')
chemistries = chemistries.rename(columns={'public_client_id':'sample_id'})
chemistries['days_in_program']=chemistries['days_in_program'].astype(float)

arivale=pd.merge_asof(
    arivale.sort_values(by='days_in_program'), 
    chemistries[chemistries.sample_id.isin(arivale.sample_id)].sort_values(by='days_in_program'), 
    by='sample_id', on='days_in_program', direction='nearest')

arivale.head()


,Unnamed: 0.1,id,simpson,inv_simpson,sample_id,days_in_program,sex,age,BMI_CALC,HEIGHT_CALC,height_cm,assessment:digestion:bowel-movements:enum,bowel_movement_frequency,vegetable_frequency,b_vegetable_frequency,LDL-CHOL CALCULATION,CRP HIGH SENSITIVITY,GLYCOHEMOGLOBIN A1C
0,5216,AV15-4372|AKE035,0.973573,37.839776,01602320,-2.0,M,47.0,37.229730,74.0,187.96,(3) 1-3 times daily,3.0,1.0,low,151.0,6.54,5.9
1,5081,AV15-3716|AKE025,0.977155,43.774041,01228476,0.0,F,50.0,22.462722,65.0,165.10,(3) 1-3 times daily,3.0,4.0,high,87.0,0.16,5.9
2,1277,22001612562257|GFM-1079-006,0.982218,56.237055,01360278,1.0,F,43.0,20.938965,64.0,162.56,(3) 1-3 times daily,3.0,4.0,high,82.0,0.22,5.6
3,4831,AV15-2608|AKE034,0.980011,50.027191,01788725,1.0,M,32.0,27.006658,69.0,175.26,(2) 3-6 times per week,2.0,4.0,high,137.0,1.12,5.5
4,4037,AV15-1295|AKE009,0.979111,47.871679,01421882,1.0,F,19.0,23.793846,65.0,165.10,NaN,NaN,2.0,low,91.0,0.20,5.6


In [5]:
# Remove individuals with abx use
abx = get_frozen_snapshot('assessments_medications')[['public_client_id', 'days_in_program', 'meds_antibiotics_last_3_months']].rename(columns = {'public_client_id':'sample_id'})
                                                                                                                                                  
arivale=pd.merge_asof(
    arivale.sort_values(by='days_in_program'), 
    abx[abx.sample_id.isin(arivale.sample_id)].sort_values(by='days_in_program'), 
    by='sample_id', on='days_in_program', direction='nearest')

arivale = arivale[arivale['meds_antibiotics_last_3_months']=='No']
arivale.head()


,Unnamed: 0.1,id,simpson,inv_simpson,sample_id,days_in_program,sex,age,BMI_CALC,HEIGHT_CALC,height_cm,assessment:digestion:bowel-movements:enum,bowel_movement_frequency,vegetable_frequency,b_vegetable_frequency,LDL-CHOL CALCULATION,CRP HIGH SENSITIVITY,GLYCOHEMOGLOBIN A1C,meds_antibiotics_last_3_months
0,5216,AV15-4372|AKE035,0.973573,37.839776,01602320,-2.0,M,47.0,37.229730,74.0,187.96,(3) 1-3 times daily,3.0,1.0,low,151.0,6.54,5.9,No
1,5081,AV15-3716|AKE025,0.977155,43.774041,01228476,0.0,F,50.0,22.462722,65.0,165.10,(3) 1-3 times daily,3.0,4.0,high,87.0,0.16,5.9,No
2,5104,AV15-3759|AKE025,0.918389,12.253256,01250245,1.0,M,32.0,46.955261,69.0,175.26,(3) 1-3 times daily,3.0,3.0,high,NaN,NaN,NaN,No
3,5150,AV15-3840|AKE025,0.957654,23.615008,01602518,1.0,F,49.0,34.154541,64.0,162.56,(3) 1-3 times daily,3.0,2.0,low,95.0,5.88,6.2,No
4,5020,AV15-3636|AKE025,0.981430,53.849198,01447446,1.0,F,56.0,21.950816,70.0,177.80,(2) 3-6 times per week,2.0,3.0,high,137.0,0.61,5.4,No


In [6]:
# Clean dataframe
arivale=arivale.sort_values(by='days_in_program').drop_duplicates(subset='sample_id', keep='first')
arivale=arivale[['sample_id', 'age', 'sex', 'BMI_CALC', 'bowel_movement_frequency', 'height_cm', 'inv_simpson','vegetable_frequency', 'b_vegetable_frequency', 'LDL-CHOL CALCULATION', 'CRP HIGH SENSITIVITY', 'GLYCOHEMOGLOBIN A1C']].dropna()

In [7]:
# Rename variables for regression
arivale = arivale.rename(columns = {'LDL-CHOL CALCULATION':'LDL', 'CRP HIGH SENSITIVITY':'CRP', 'GLYCOHEMOGLOBIN A1C':'HBA1C'})
arivale.head()

,sample_id,age,sex,BMI_CALC,bowel_movement_frequency,height_cm,inv_simpson,vegetable_frequency,b_vegetable_frequency,LDL,CRP,HBA1C
0,01602320,47.0,M,37.229730,3.0,187.96,37.839776,1.0,low,151.0,6.54,5.9
1,01228476,50.0,F,22.462722,3.0,165.10,43.774041,4.0,high,87.0,0.16,5.9
11,01360278,43.0,F,20.938965,3.0,162.56,56.237055,4.0,high,82.0,0.22,5.6
8,01048590,47.0,F,22.459158,3.0,160.02,28.621199,3.0,high,76.0,0.68,5.5
7,01826852,35.0,F,24.541709,3.0,154.94,7.976489,4.0,high,93.0,1.94,6.0


In [8]:
# Log transform Simpson's Diversity and height
features = ['inv_simpson', 'height_cm']
arivale[features] = arivale[features].apply(lambda x: np.log(x))

# # Standardize data
features=['age', 'BMI_CALC', 'height_cm', 'inv_simpson', 'LDL', 'CRP', 'HBA1C'] 
arivale[features]=arivale[features].apply(lambda x:(x-x.mean()) / x.std())


In [9]:
#### Define dictionary with each covariate and formulas excluding each to determine individual R2 for every covar
covars = ['C(sex)', 'age', 'BMI_CALC', 'bowel_movement_frequency', 'vegetable_frequency', 'height_cm', 'height_cm:vegetable_frequency', 'LDL', 'CRP', 'HBA1C']
d_formula=dict()
d_formula[0] = {
    'formula':'inv_simpson ~ age + C(sex) + BMI_CALC + bowel_movement_frequency + vegetable_frequency + height_cm + height_cm:vegetable_frequency + LDL + CRP + HBA1C',
    'excluded':'none',
    # 'covariates':list(map(lambda x: x.replace('C(sex)', 'C(sex)[T.M]').replace('C(vegetable_frequency)','C(vegetable_frequency)[T.low]'), covars))}
    'covariates':list(map(lambda x: x.replace('C(sex)', 'C(sex)[T.M]'), covars))}
d_formula[1] = {
    'formula':'inv_simpson ~ height_cm',
    'excluded':'all but height',
    'covariates':float('NaN')}

for i in range(0,len(covars)):
    excluded=covars[i]
    temp_covars=[i for i in covars if i != excluded]
    formula=' + '.join(temp_covars)
    formula = 'inv_simpson ~ '+formula
    temp_covars=list(map(lambda x: x.replace('C(sex)', 'C(sex)[T.M]'),temp_covars))
    
    d_formula[i+2] = {
        'formula':formula,
        'excluded':excluded,
        'covariates':temp_covars}

# Formula for determining indivudal R2 for every covariate
def find_covar_rsq(dataframe, f_dict):
    big_squared=pd.DataFrame()
    
    for i in range(0, len(f_dict)): # for every formula combination in my dictionary 
        if i==0: # first entry in the dict is the full formula 
            results = smf.ols(formula=f_dict[i]['formula'], data=dataframe).fit() 
            ref_r=results.rsquared # this is going to be used to derive r_squared of indiviudal covariates
            betas=pd.DataFrame(results.params).reset_index().rename(columns={'index':'covar', 0:'Beta'}) # get beta coefficients
            betas['formula']=f_dict[i]['formula'] # record the formula used
            betas['rsquared_model']=results.rsquared # get the rsquared of the model
            ps = pd.DataFrame(results.pvalues.loc[f_dict[i]['covariates']]).reset_index().rename(columns={0:'p_covar', 'index':'covar'}) # get pvalues for each covar
            betas=pd.merge(betas, ps, on = 'covar') #merge ps with beta coefficients for each covar
            
            ### F-test to compare model with and without height as a covar
            model_reduced = smf.ols(formula = 'inv_simpson ~ age + C(sex) + BMI_CALC + bowel_movement_frequency + vegetable_frequency', data=dataframe).fit()
            F_test = anova_lm(model_reduced, results)
            betas['F']=F_test.loc[1,'F']
            betas['F_pval']=F_test.loc[1,'Pr(>F)']

        
        elif i==1: # the second entry in my dict is just simpson vs height
            # like before, get beta coefficient and r^2 values
            results=smf.ols(formula=f_dict[i]['formula'], data=dataframe).fit()
            temp=pd.DataFrame(results.params).reset_index().rename(columns={'index':'covar', 0:'Beta'})
            temp['formula']=f_dict[i]['formula']
            temp['rsquared_model']=results.rsquared
            temp['p_covar']=results.pvalues['height_cm']
            
            F_test = anova_lm(results)
            temp['F']=F_test.loc['height_cm','F']
            temp['F_pval']=F_test.loc['height_cm','PR(>F)']
            betas=pd.concat([betas, temp])
        
        else: # for every other formula, we excluded one covar so we can derive the r^2 for indiviudal covars
            results = smf.ols(formula=f_dict[i]['formula'], data=dataframe).fit() #ols with the formula[i]
            squared_df=pd.DataFrame(columns=['covar', 'rsquared_covar', 'formula', 'rsquared_model'])
            squared_df.loc[i,'covar']=f_dict[i]['excluded'] # the excluded covar is the one we are getting an r^2 value for
            squared_df.loc[i,'rsquared_covar']=ref_r-results.rsquared
            squared_df.loc[i,'formula']=f_dict[i]['formula']
            squared_df.loc[i,'rsquared_model']=ref_r
            big_squared=pd.concat([big_squared, squared_df])
    
    #Now we have everything we need to make one big df with my regression info 
    big_squared.loc[2,'covar']='C(sex)[T.M]' #so I can merge my dfs on covars
    # big_squared.loc[6,'covar']='C(vegetable_frequency)[T.low]'
    complete=pd.merge(big_squared[['covar', 'rsquared_covar', 'rsquared_model']], \
                      betas[['covar', 'Beta', 'formula', 'rsquared_model', 'p_covar', 'F', 'F_pval']], \
                      on=['covar', 'rsquared_model'], how='right')
    return(complete, F_test)

arivale_results, arivale_F= find_covar_rsq(arivale, d_formula)


In [10]:
arivale_results

,covar,rsquared_covar,rsquared_model,Beta,formula,p_covar,F,F_pval
0,C(sex)[T.M],0.002542,0.06477,-1.521272e-01,inv_simpson ~ age + C(sex) + BMI_CALC + bowel_...,7.777428e-03,10.95402,1.875719e-10
1,age,0.007594,0.06477,9.017540e-02,inv_simpson ~ age + C(sex) + BMI_CALC + bowel_...,4.348971e-06,10.95402,1.875719e-10
2,BMI_CALC,0.008416,0.06477,-1.072950e-01,inv_simpson ~ age + C(sex) + BMI_CALC + bowel_...,1.330580e-06,10.95402,1.875719e-10
3,bowel_movement_frequency,0.009784,0.06477,-1.874314e-01,inv_simpson ~ age + C(sex) + BMI_CALC + bowel_...,1.874410e-07,10.95402,1.875719e-10
4,vegetable_frequency,0.001516,0.06477,4.605855e-02,inv_simpson ~ age + C(sex) + BMI_CALC + bowel_...,3.977037e-02,10.95402,1.875719e-10
5,height_cm,0.003014,0.06477,1.919596e-01,inv_simpson ~ age + C(sex) + BMI_CALC + bowel_...,3.760615e-03,10.95402,1.875719e-10
6,height_cm:vegetable_frequency,0.000168,0.06477,-1.507022e-02,inv_simpson ~ age + C(sex) + BMI_CALC + bowel_...,4.933511e-01,10.95402,1.875719e-10
7,LDL,0.000121,0.06477,-1.127072e-02,inv_simpson ~ age + C(sex) + BMI_CALC + bowel_...,5.609913e-01,10.95402,1.875719e-10
8,CRP,0.002118,0.06477,-5.228354e-02,inv_simpson ~ age + C(sex) + BMI_CALC + bowel_...,1.510933e-02,10.95402,1.875719e-10
9,HBA1C,0.004648,0.06477,-7.384013e-02,inv_simpson ~ age + C(sex) + BMI_CALC + bowel_...,3.223246e-04,10.95402,1.875719e-10


In [11]:
ari_ols=smf.ols(formula='inv_simpson ~ age + C(sex) + BMI_CALC + bowel_movement_frequency + vegetable_frequency + height_cm + vegetable_frequency:height_cm + LDL + CRP + HBA1C', data=arivale).fit()
print(ari_ols.summary())

                            OLS Regression Results                            
Dep. Variable:            inv_simpson   R-squared:                       0.065
Model:                            OLS   Adj. R-squared:                  0.061
Method:                 Least Squares   F-statistic:                     18.08
Date:                Mon, 24 Jun 2024   Prob (F-statistic):           2.51e-32
Time:                        14:23:47   Log-Likelihood:                -3630.8
No. Observations:                2621   AIC:                             7284.
Df Residuals:                    2610   BIC:                             7348.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     